# Create spark session

In [18]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType, DateType
from pyspark.sql import functions as F

my_conf = SparkConf()
my_conf.set("spark.app.name", "My Application")
my_conf.set("spark.ui.port", "4050")

spark = SparkSession \
            .builder \
            .config(conf=my_conf) \
            .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## 1. Create a Python List as a dataset

In [3]:
# Load the data in a Python List 

sample_data = [(1,"2013-07-25",11599,"CLOSED"),
(2,"2013-07-25",256,"PENDING_PAYMENT"),
(3,"2013-07-25",11599,"COMPLETE"),
(4,"2013-07-25",8827,"CLOSED")]

## 2 . Create a DF with column names 

In [15]:
# Create a DF from this List 

df = spark.createDataFrame(sample_data, ['order_id', "order_date", "customer_id", "order_status"])

# One more alternate way 
# df = spark.createDataFrame(sample_data).toDF('order_id', "order_date", "customer_id", "order_status")

In [16]:
df.show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      11599|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
+--------+----------+-----------+---------------+



In [17]:
df.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- order_status: string (nullable = true)



## 3. Convert the time stamp 

In [21]:
from datetime import date, datetime


In [36]:
new_df = df.withColumn(
    'order_date', F.unix_timestamp(df.order_date))

new_df.show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|      null|      11599|         CLOSED|
|       2|      null|        256|PENDING_PAYMENT|
|       3|      null|      11599|       COMPLETE|
|       4|      null|       8827|         CLOSED|
+--------+----------+-----------+---------------+



## 4. Adding a new column, dropping duplicate, droping `order_id` and sorting by `order_date`

In [49]:
new_df = df \
            .withColumn('order_date_new', F.unix_timestamp(F.col('order_date'),'yyyy-MM-dd')) \
            .withColumn('unique_id', F.monotonically_increasing_id()) \
            .drop_duplicates(['order_date', 'customer_id']) \
            .drop('order_id') \
            .sort('order_date')


new_df.show()

+----------+-----------+---------------+--------------+-----------+
|order_date|customer_id|   order_status|order_date_new|  unique_id|
+----------+-----------+---------------+--------------+-----------+
|2013-07-25|        256|PENDING_PAYMENT|    1374710400|17179869184|
|2013-07-25|       8827|         CLOSED|    1374710400|34359738368|
|2013-07-25|      11599|         CLOSED|    1374710400| 8589934592|
+----------+-----------+---------------+--------------+-----------+



In [40]:
df.show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|2013-07-25|      11599|         CLOSED|
|       2|2013-07-25|        256|PENDING_PAYMENT|
|       3|2013-07-25|      11599|       COMPLETE|
|       4|2013-07-25|       8827|         CLOSED|
+--------+----------+-----------+---------------+



# Aggregation 

## 1. Simple Aggregation

In [50]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType, DateType
from pyspark.sql import functions as F

my_conf = SparkConf()
my_conf.set("spark.app.name", "My Application")
my_conf.set("spark.ui.port", "4050")

spark = SparkSession \
            .builder \
            .config(conf=my_conf) \
            .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [51]:
# Read the data from the CSV 
DATASET_PATH = "s3://data-engg-suman/dataset/order_data.csv"

In [52]:
df = spark \
        .read \
        .format('csv') \
        .option('header', True) \
        .option('inferSchema', True) \
        .option('path', DATASET_PATH) \
        .load()

In [54]:
df.show(4)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO...|       6|01-12-2010 9.37|     2.95|     14688|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 4 rows



In [65]:
# Using Column Object 

df.select(F.count('*').alias('row_count'), \
        F.sum('Quantity').alias('total_quantity'), \
        F.avg('UnitPrice').alias('avg_unit_price'), \
        F.countDistinct('InvoiceNo').alias('distinct_invoice') \
    ).show()

+---------+--------------+---------------+----------------+
|row_count|total_quantity| avg_unit_price|distinct_invoice|
+---------+--------------+---------------+----------------+
|   541782|       5175855|4.6115653233219|           25858|
+---------+--------------+---------------+----------------+



In [72]:
# Using Spark SQL

df.createOrReplaceTempView('sales')

spark.sql('SELECT COUNT(*) AS row_count, \
                  SUM(Quantity) AS total_quantity, \
                  AVG(UnitPrice) AS avg_unit_price, \
                  COUNT(DISTINCT InvoiceNo) AS distinct_invoice\
            FROM sales').show()

+---------+--------------+---------------+----------------+
|row_count|total_quantity| avg_unit_price|distinct_invoice|
+---------+--------------+---------------+----------------+
|   541782|       5175855|4.6115653233219|           25858|
+---------+--------------+---------------+----------------+



## 2. Grouping Aggregation

In [73]:
df.show(5)

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536378|     null|PACK OF 60 DINOSA...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|     null|PACK OF 60 PINK P...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|    84991|60 TEATIME FAIRY ...|      24|01-12-2010 9.37|     0.55|     14688|United Kingdom|
|   536378|   84519A|TOMATO CHARLIE+LO...|       6|01-12-2010 9.37|     2.95|     14688|United Kingdom|
|   536378|   85183B|CHARLIE & LOLA WA...|      48|01-12-2010 9.37|     1.25|     14688|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
only showing top 5 rows



In [86]:
# Group the data based on Country and Invoice Number and we need the total quantity per country (using Column Object Exression)

df.groupBy(['Country', 'InvoiceNo']) \
    .agg(F.sum('Quantity').alias('Total_Quantity'), \
         F.sum(F.col('Quantity') * F.col('UnitPrice')).alias('InvoiceValue')) \
    .show()

+--------------+---------+--------------+------------------+
|       Country|InvoiceNo|Total_Quantity|      InvoiceValue|
+--------------+---------+--------------+------------------+
|United Kingdom|   536508|           216|            155.52|
|        Cyprus|   544574|           173|            320.69|
|United Kingdom|   546372|            88|349.20000000000005|
|United Kingdom|   536627|            64|306.20000000000005|
|United Kingdom|   537224|           700|1415.9700000000003|
|        Israel|  C539037|           -56|           -227.44|
|United Kingdom|   540945|             1|              0.85|
|United Kingdom|   546318|           100|254.99999999999997|
|United Kingdom|   537855|            95|263.24999999999994|
|United Kingdom|   538090|          -723|               0.0|
|United Kingdom|   539040|            43|            171.05|
|United Kingdom|   540465|            78|              83.7|
|         Spain|   540785|          1008|           3417.12|
|United Kingdom|  C54183

In [82]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [90]:
# Using Spark SQL 

df.createGlobalTempView('sales')

spark.sql('select Country,InvoiceNo, SUM(Quantity) as Total_Quantity, SUM(Quantity * UnitPrice) as InvoiceValue from sales GROUP BY Country,InvoiceNo').show()

+--------------+---------+--------------+------------------+
|       Country|InvoiceNo|Total_Quantity|      InvoiceValue|
+--------------+---------+--------------+------------------+
|United Kingdom|   536508|           216|            155.52|
|        Cyprus|   544574|           173|            320.69|
|United Kingdom|   546372|            88|349.20000000000005|
|United Kingdom|   536627|            64|306.20000000000005|
|United Kingdom|   537224|           700|1415.9700000000003|
|        Israel|  C539037|           -56|           -227.44|
|United Kingdom|   540945|             1|              0.85|
|United Kingdom|   546318|           100|254.99999999999997|
|United Kingdom|   537855|            95|263.24999999999994|
|United Kingdom|   538090|          -723|               0.0|
|United Kingdom|   539040|            43|            171.05|
|United Kingdom|   540465|            78|              83.7|
|         Spain|   540785|          1008|           3417.12|
|United Kingdom|  C54183

## 3. Window Aggregations 

In [91]:
# Read the data from the CSV 
DATASET_PATH = "s3://data-engg-suman/dataset/windowdata.csv"

df = spark \
        .read \
        .format('csv') \
        .option('header', False) \
        .option('inferSchema', True) \
        .option('path', DATASET_PATH) \
        .load()

In [95]:
df1 = df.toDF('country', 'weeknum', 'numinvoices', 'totalquantity', 'invoicevalue')

In [100]:
df1.show(2)

+-------+-------+-----------+-------------+------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|
+-------+-------+-----------+-------------+------------+
|  Spain|     49|          1|           67|      174.72|
|Germany|     48|         11|         1795|     3309.75|
+-------+-------+-----------+-------------+------------+
only showing top 2 rows



In [101]:
from pyspark.sql import Window

mywindow = Window.partitionBy('country') \
                 .orderBy('weeknum') \
                 .rowsBetween(Window.unboundedPreceding, Window.currentRow)

final_df = df1.withColumn('Running_amount', F.sum('invoicevalue').over(mywindow))
final_df.show()

+---------------+-------+-----------+-------------+------------+------------------+
|        country|weeknum|numinvoices|totalquantity|invoicevalue|    Running_amount|
+---------------+-------+-----------+-------------+------------+------------------+
|      Australia|     48|          1|          107|      358.25|            358.25|
|      Australia|     49|          1|          214|       258.9|            617.15|
|      Australia|     50|          2|          133|      387.95|1005.0999999999999|
|        Austria|     50|          2|            3|      257.04|            257.04|
|        Bahrain|     51|          1|           54|      205.74|            205.74|
|        Belgium|     48|          1|          528|       346.1|             346.1|
|        Belgium|     50|          2|          285|      625.16|            971.26|
|        Belgium|     51|          2|          942|      838.65|1809.9099999999999|
|Channel Islands|     49|          1|           80|      363.53|            

# JOIN Operations

In [134]:
spark.stop()

In [136]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "My Application")
my_conf.set("spark.ui.port", "4051")

spark = SparkSession \
            .builder \
            .config(conf=my_conf) \
            .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [137]:
# Read the data from the CSV 
DATASET_PATH = "s3://data-engg-suman/dataset/customers.csv"

df_customers = spark \
        .read \
        .format('csv') \
        .option('header', True) \
        .option('inferSchema', True) \
        .option('path', DATASET_PATH) \
        .load()

In [138]:
# Read the data from the CSV 
DATASET_PATH = "s3://data-engg-suman/dataset/orders.csv"

df_orders = spark \
        .read \
        .format('csv') \
        .option('header', True) \
        .option('inferSchema', True) \
        .option('path', DATASET_PATH) \
        .load()

In [139]:
df_customers.show(3)

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|             725|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------

In [140]:
df_orders.show(3)

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
+--------+-------------------+-----------------+---------------+
only showing top 3 rows



In [147]:
joining_condition = df_customers.customer_id == df_orders.order_customer_id
joined_df = df_orders.join(df_customers, joining_condition, 'inner')

In [148]:
joined_df.show(3)

+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|      11599|          Mary|        Malone|     XXXXXXXXX|        XXXXXXXXX|8708 Indian Horse...|      Hickory|            NC|           28601|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|        256|         David|     Rodriguez|     XXXXXXXXX|        XXXXXXXXX|7605 Tawny Horse ...|      Chicag

In [144]:
joining_condition = df_customers.customer_id == df_orders.order_customer_id
joined_df = df_orders.join(df_customers, joining_condition, 'outer').withColumn('order_id', F.expr('coalesce(order_id, -1)'))
joined_df.show()

+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|order_id|         order_date|order_customer_id|   order_status|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+--------+-------------------+-----------------+---------------+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|   68884|               null|             null|       COMPLETE|       null|          null|          null|          null|             null|                null|         null|          null|            null|
|      -1|               null|             null|           null|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsvill

In [149]:
spark.stop()